## Import necessary libraries

In [1]:
# #1 Import necessary libraries
import numpy as np
import tensorflow as tf
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score


## Load and preprocess dataset

In [2]:

# TensorFlow
(X_train_tf, y_train_tf), (X_test_tf, y_test_tf) = tf.keras.datasets.mnist.load_data()
X_train_tf, X_test_tf = X_train_tf / 255.0, X_test_tf / 255.0


In [3]:

# PyTorch
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


Failed to download (trying next):
<urlopen error [Errno 11001] getaddrinfo failed>



100%|██████████| 9912422/9912422 [00:09<00:00, 1043421.93it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 154520.65it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:02<00:00, 652176.98it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 756123.39it/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



## Define and build models

In [4]:

# TensorFlow
model_tf = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])


c:\Users\New User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [5]:

# PyTorch
class ModelPT(nn.Module):
    def __init__(self):
        super(ModelPT, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

model_pt = ModelPT()


## Define Loss Function, Optimizer and Metric

In [6]:

# TensorFlow
loss_fn_tf = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer_tf = tf.keras.optimizers.Adam(learning_rate=0.001)
accuracy_tf = tf.keras.metrics.SparseCategoricalAccuracy()


In [7]:

# PyTorch
loss_fn_pt = nn.CrossEntropyLoss()
optimizer_pt = optim.Adam(model_pt.parameters(), lr=0.001)


## Train models

In [8]:

# TensorFlow
model_tf.compile(optimizer=optimizer_tf, loss=loss_fn_tf, metrics=[accuracy_tf])
model_tf.fit(X_train_tf, y_train_tf, epochs=100)


Epoch 1/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 0.4867 - sparse_categorical_accuracy: 0.8584
Epoch 2/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.1552 - sparse_categorical_accuracy: 0.9547
Epoch 3/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.1120 - sparse_categorical_accuracy: 0.9661
Epoch 4/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0855 - sparse_categorical_accuracy: 0.9747
Epoch 5/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0712 - sparse_categorical_accuracy: 0.9777
Epoch 6/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0623 - sparse_categorical_accuracy: 0.9806
Epoch 7/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0543 - sparse_categorical_accuracy: 0.9824
Epoch 8/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 0.0518 - sparse_categorical_accuracy: 0.9828
Epoch 9/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - loss: 0.0470 - sparse_categorical_accuracy: 0.9847
Epoch 10/100
1875/1875 ━━━

In [9]:

# PyTorch
for epoch in range(100):
    for data in trainloader:
        inputs, labels = data
        optimizer_pt.zero_grad()
        outputs = model_pt(inputs)
        loss = loss_fn_pt(outputs, labels)
        loss.backward()
        optimizer_pt.step()


## Evaluate models

In [10]:

# TensorFlow
_, accuracy_tf = model_tf.evaluate(X_test_tf, y_test_tf)
print("Accuracy (TensorFlow):", accuracy_tf)


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1966 - sparse_categorical_accuracy: 0.9733
Accuracy (TensorFlow): 0.9775000214576721


In [11]:

# PyTorch
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model_pt(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print("Accuracy (PyTorch):", correct / total)


Accuracy (PyTorch): 0.9636
